In [ ]:
#
# Tests the ability to use springs layout selectively on subgraphs
# ... so, if you keep the placement of the nodes from an initial all-nodes layout
# ... then you can update the placement of a subset of the nodes
#
# However... because that's not the only transform done to make this all work
# (think about the treemap component placement), then there's always a scaling factor
# involved... and that breaks the ability to apply the spring layout to a subset of 
# nodes...
#

import polars   as pl
import networkx as nx
import random
import copy
import rtsvg
import linknode_graph_patterns
from rtsvg.polars_spring_layout import PolarsSpringLayout
rt   = rtsvg.RACETrack()
g    = linknode_graph_patterns.LinkNodeGraphPatterns().__pattern_mesh__()
pos  = PolarsSpringLayout(g).results()
_lu_ = {'fm':[], 'to':[]}
for n in g.nodes():
    for nbor in g.neighbors(n): _lu_['fm'].append(n), _lu_['to'].append(nbor)
df        = pl.DataFrame(_lu_)
_relates_ = [('fm','to')]
_to_fix_  = {'node_0_5', 'node_7_4', 'node_8_6', 'node_5_8'}
_colors_  = {n:'red' for n in _to_fix_}
new_pos   = copy.deepcopy(pos)
for n in _to_fix_: new_pos[n] = (new_pos[n][0] + 100*(random.random()-0.5), new_pos[n][1] + 100*(random.random()-0.5))
new_new_pos = copy.deepcopy(new_pos)
new_new_pos = PolarsSpringLayout(g, pos=new_new_pos, static_nodes=set(g.nodes()) - _to_fix_).results()
params      = {'w':384, 'h':384, 'node_color':_colors_}
rt.tile([rt.link(df, _relates_, pos,         **params),
         rt.link(df, _relates_, new_pos,     **params),
         rt.link(df, _relates_, new_new_pos, **params)], spacer=10)

In [ ]:
#
# So... maybe it's possible to rescale the graph based on the expected distances
# ... in this case (mesh graph), all edges should be 1.0 ... so... how to take
# ... a random set of positions & then rescale them?
#
_d_sum_, _d_samples_, _rms_sum_ = 0.0, 0, 0.0
for n in g.nodes():
    for nbor in g.neighbors(n):
        _w_                  =  g[n][nbor]['weight']
        _xy_n_, _xy_nbor_    =  pos[n], pos[nbor]
        _d_                  =  rt.segmentLength((_xy_n_, _xy_nbor_))
        _w_diff_             =  _w_ - _d_
        _rms_sum_            += _w_diff_**2
        _d_sum_, _d_samples_ = _d_sum_ + _d_, _d_samples_ + 1
print(_d_sum_, _d_samples_, _d_sum_ / _d_samples_) # 359.24853900508407 288 1.2473907604343197 -- example from the mesh graph

In [ ]:
#
# Grid Search Version of This Optimization
#
def rms(pos, g, x_scale, y_scale):
    _d_sum_, _d_samples_, _rms_sum_ = 0.0, 0, 0.0
    for n in g.nodes():
        for nbor in g.neighbors(n):
            _w_                  =  g[n][nbor]['weight']
            _xy_n_, _xy_nbor_    =  (pos[n][0]*x_scale, pos[n][1]*y_scale), (pos[nbor][0]*x_scale, pos[nbor][1]*y_scale)
            _d_                  =  rt.segmentLength((_xy_n_, _xy_nbor_))
            _w_diff_             =  _w_ - _d_
            _rms_sum_            += _w_diff_**2
            _d_sum_, _d_samples_ = _d_sum_ + _d_, _d_samples_ + 1
    _rms_ = _rms_sum_ / _d_samples_
    _rms_ = _rms_**0.5
    return _rms_

_lu_ = {'x_scale':[], 'y_scale':[], 'rms':[]}
_inc_     = 0.05
_x_scale_ = 0.5
while _x_scale_ < 1.5:
    _y_scale_ = 0.5
    while _y_scale_ < 1.5:
        _rms_ = rms(pos, g, _x_scale_, _y_scale_)
        _lu_['x_scale'].append(_x_scale_)
        _lu_['y_scale'].append(_y_scale_)
        _lu_['rms'].append(_rms_)
        _y_scale_ += _inc_
    _x_scale_ += _inc_


rt.tile([rt.xy(pl.DataFrame(_lu_), x_field='x_scale', y_field='rms', line_groupby_field='y_scale', color_by='y_scale', dot_size=None),
         rt.xy(pl.DataFrame(_lu_), x_field='y_scale', y_field='rms', line_groupby_field='x_scale', color_by='x_scale', dot_size=None)], spacer=10)

In [ ]:
from math import sqrt
_lu_ = {'n':[], 'nbor':[], 'w':[], 'n_x':[], 'nbor_x':[], 'n_y':[], 'nbor_y':[]}
for n in g.nodes():
    for nbor in g.neighbors(n):
        _w_                  =  g[n][nbor]['weight']
        _xy_n_, _xy_nbor_    =  pos[n], pos[nbor]
        _lu_['n'].append(n),                 _lu_['nbor'].append(nbor),             _lu_['w'].append(_w_)
        _lu_['n_x'].append(_xy_n_[0]),       _lu_['n_y'].append(_xy_n_[1])
        _lu_['nbor_x'].append(_xy_nbor_[0]), _lu_['nbor_y'].append(_xy_nbor_[1])
df_for_rms = pl.DataFrame(_lu_)
def rmsWithPolars(x_scale, y_scale):
    __n_x__, __n_y__       = pl.col('n_x'), pl.col('n_y')
    __nbor_x__, __nbor_y__ = pl.col('nbor_x'), pl.col('nbor_y')
    _df_ = df_for_rms.with_columns(((__n_x__*x_scale - __nbor_x__*x_scale)**2 + (__n_y__*y_scale - __nbor_y__*y_scale)**2).alias('x2'))
    _df_ = _df_.with_columns((pl.col('x2')**0.5).alias('d'))
    _df_ = _df_.with_columns(((df_for_rms['w'] - _df_['d'])**2).alias('rms'))
    return sqrt(_df_['rms'].sum()/len(_df_))
rmsWithPolars(1.0, 1.0), rms(pos, g, 1.0, 1.0)

In [ ]:
%%timeit
rmsWithPolars(1.0, 1.0)

In [ ]:
%%timeit
rms(pos, g, 1.0, 1.0)